In [4]:
! pip install Biopython


In [1]:

#load needed libraries
from Bio import Entrez 
from xmljson import yahoo as yh
from xml.etree.ElementTree import fromstring
import xmltodict


In [2]:
def get_info():
    email = input('Please provide email address: ')
    return email

In [3]:
def get_abstract(term_ids):
    fetch_handle = Entrez.efetch(db='pubmed', id=term_ids, retmode='xml', retmax=4000, rettype='abstract')
    return fetch_handle.read()

In [4]:
def findformat_abstract(terms):
    # Create an empty dictionary - can also use dict() 
    new_abstracts = {}
    
    # require an email input to search the Entrez databases
    Entrez.email = get_info()
    
    # For each term in the LIST of term ...
    for term in terms:
        # The try block lets you test a block of code for errors.
        # and the except block lets you handle the error.
        # next, the else block lets you execute code when there is no error.

        try:
            # known - do a term search extracting the pubmed ids
            pub_handle = Entrez.esearch(db = "pubmed", term=term, retmax=4000,sort="relevance", retmode = "xml")
            pub_record = Entrez.read(pub_handle)
            pub_id = pub_record["IdList"]

            # known - get abstracts for the pubmed ids 
            abstracts= get_abstract(pub_id)
            y_abstracts = yh.data(fromstring(abstracts))

            #look at structure of data to see what data structures exists - how do we retrieve these structures
            try:
                format_abstracts = y_abstracts['PubmedArticleSet']['PubmedArticle'] #dict
                for ref_data in (format_abstracts): #list
                    ref_medline_citation = ref_data['MedlineCitation'] #dict
                    ref_pubmeddata = ref_data['PubmedData']['ArticleIdList']['ArticleId']
                    pmid = ref_medline_citation['PMID']['content']
                    journal = ref_medline_citation['Article']['Journal']['Title']
                    title = ref_medline_citation['Article']['ArticleTitle']
                    authors = ref_medline_citation['Article']['AuthorList']['Author']
                    try:
                        pmc = [each_id['content'] for each_id in ref_pubmeddata if each_id['IdType']=='pmc']
                        doi = [each_id['content'] for each_id in ref_pubmeddata if each_id['IdType']=='doi']
                    except:
                        pass
                    names = []
                    if not isinstance(authors,list):
                        authors=[authors,]
                    for item in authors:
                        names.append(item.get('LastName',"")+ ' '+ item.get('ForeName',"") + ' ' + item.get('Initials',""))
                    mesh = []
                    try:
                        headings = ref_medline_citation['MeshHeadingList']['MeshHeading']
                        for item2 in headings:
                            mesh.append(item2['DescriptorName']['content'])
                    except KeyError:
                        mesh = "NONE"
                    try:
                        abstract = ref_medline_citation['Article']['Abstract']['AbstractText']
                    except KeyError:
                        abstract = "NONE"
                    try:
                        month = ref_medline_citation['Article']['ArticleDate']['Month']
                        day = ref_medline_citation['Article']['ArticleDate']['Day']
                        year = ref_medline_citation['Article']['ArticleDate']['Year']
                        date =  month,day,year
                    except KeyError:
                        date = "NONE"
                    if pmid in new_abstracts:
                         new_abstracts[pmid]['Search Terms'].append(term)
                    else:
                         new_abstracts[pmid] = {'Search Terms': [term], 'DOI': doi, 'PMC': pmc,'Date': date, 'Authors':names, 'Journal':journal, 'Title':title, 'Abstract':abstract,'MESH':mesh, 'Methods': [], 'Results':[], 'fit_main_keywords':[]}
            except (KeyError, TypeError):
                continue
        except urllib.error.HTTPError:
            time.sleep(10)
    return new_abstracts